<a href="https://colab.research.google.com/github/asarria48/ML-for-materials/blob/main/Advanced_BO_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Periodicity and symmetries in Bayesian optimization (Advanced content)

In this tutorial we will study how periodicity and symmetries in the objective function can be leveraged to reduce the number of iterations required to find a global optimum.

We will use a simple example from atomic materials simulations where the goal is to find the
optimal (lowest energy) position of a Pd atom on an Au surface. In this context Pd is referred
to as an adatom. More precisely, we will place a Pd atom on a 2x2 Au(100) slab, resulting in a
1/4 ML coverage. To keep the simulation time short we will use a simple method (Embedded Medium Theory, EMT)
to calculate the energy as a function of the position of the adatom.


---


**NOTE**: If you are not familiar with surface science, you can think of this as a purely mathematical problem where we want  to minimize a function (the energy) of three variables (the x, y, z coordinates of the adatom). This objective function is periodic, and has mirror symmetries, for two variables $(x, y)$.


In [ ]:
!pip3 install ase aalto-boss
import numpy as np

from boss.bo.bo_main import BOMain
from boss.pp.pp_main import PPMain

from ase.build import fcc100
from ase import Atoms
from ase.calculators.emt import EMT
from ase.visualize import view

Below we define a helper class that assembles the adatom system given a desired location for the adatom and
uses the EMT method to calculate the energy. The method for calculating the energy `Adatom.calc_energy` takes a
single array $x$ as argument and places the adatom at position $x_1\mathbf{a}_1 + x_2\mathbf{a}_2 + x_3\mathbf{z}$,
where $(\mathbf{a}_1, \mathbf{a}_2)$ are the lattice vectors of the primitive surface cell.

In [ ]:
class Adatom:
    def __init__(self, z_fix=None):
        """Models a Pd adatom layer on an Au surface.

        Parameters
        ----------
        z_fix: Optional[float]
            A fixed cartesian z-coordinate for the adatom, measured
            with respect to surface's uppermost atomic layer.
        """
        self.a0 = 4.056  # EMT lattice const. for Au
        self.slab = fcc100('Au', size=(2, 2, 5), a=self.a0, vacuum=7.5)
        self.adatom = Atoms('Pd')
        self.adatom.cell = self.slab.cell
        self.z_top = max(at.z for at in self.slab)
        self.calc = EMT()
        self.atoms = None
        self.z_fix = z_fix

    def place(self, x):
        """Places the adatom at the given coordinates.
        """
        x = np.squeeze(x)
        if self.z_fix is not None and len(x) == 2:
            x = np.append(x, self.z_fix)
        self.adatom[0].z = self.z_top + x[2]
        pos_scaled = self.adatom.get_scaled_positions()[0]
        self.adatom.set_scaled_positions([0.5*x[0], 0.5*x[1], pos_scaled[2]])
        self.atoms = self.slab + self.adatom

    def calc_energy(self, x):
        """Calculates the energy of the combined slab + adatom using the EMT calculator."""
        self.place(x)
        energy = self.calc.get_potential_energy(self.atoms)
        return energy

Let's start by investigating the problem in 2D by initializing the `Adatom` class with the `z_fix` argument
which fixes the adatom's z-coordinate so we only need to care about the (x, y) coordinates. We can then run
BO with BOSS as usual by passing the `Adatom.calc_energy` method as the objective function

In [ ]:
adatom = Adatom(z_fix=2.5)

bo = BOMain(
    adatom.calc_energy,
    bounds=[[0., 1.], [0., 1.]],
    kernel=['rbf', 'rbf'],
    initpts=4,
    iterpts=20,
    acqfn_name='lcb',
)
res = bo.run()

pp = PPMain(res, pp_models=True, pp_acq_funcs=True)
pp.run()

# We can also visualize the minimum energy configuration
x_glmin = res.select("x_glmin", -1)
y_glmin = res.select("mu_glmin", -1)
print("Best adsorption energy found: %0.3f eV" %y_glmin)
adatom.place(x_glmin)
view(adatom.atoms, viewer='x3d')

Here, we have used the RBF kernel, which assumes that the objective is smooth but doesn't otherwise constrain the objective.
We know, however (due to the structure of the gold surface), that the the objective is periodic in $(x, y)$ with period 1, coinciding
with the bounds. This knowledge can be encoded by into the GP by using the so-called standard periodic kernel
$$ k_{\mathrm{per}}(\mathbf{x}, \mathbf{x}') = \sigma^2 \exp\left(\frac{2\sin^2\left(\left(\pi\left\|\mathbf{x} - \mathbf{x}'\right\|/p\right)\right)}{l^2}\right).$$
Here, $\sigma^2, l$ are the variance and lengthscale hyperparameters and $p$ is a (typically fixed) period.
To use the standard periodic kernel with BOSS, we simply interchange the `rbf` keyword for `stdp`. Repeat the previous BO run with periodic kernels and compare
the model plots generated by the post-processing tool, can you notice any differences? Before running the new optimization you might want to rename the post-processing
folder from the previous run so it doesn't get overwritten.

Let us also allow z variable to change, 1-2.5 angstrom would be a reasonable range. Note, that the z variable is not periodic, so we must use an RBF kernel here.

In [ ]:
# Switching on periodic boundary conditions & adding Z variation.
# 1-2.5 angstrom above the surface is a good height
adatom = Adatom()

bo = BOMain(
    adatom.calc_energy,
    bounds=[[0., 1.], [0., 1.], [1, 2.5]],
    kernel=['stdp', 'stdp', 'rbf'],
    initpts=4,
    iterpts=20,
    acqfn_name='elcb',
)
res = bo.run()

pp = PPMain(res, pp_models=True, pp_acq_funcs=True)
pp.run()

# We can also view the minimum energy configuration
x_glmin = res.select("x_glmin", -1)
y_glmin = res.select("mu_glmin", -1)
print("Best adsorption energy found: %0.3f eV" %y_glmin)
adatom.place(x_glmin)
view(adatom.atoms, viewer='x3d')


Best adsorption energy found: -6.022 eV


If you are familiar with crystal structures, you might find that the minimum energy configuration found above doesn't match your expectations. Indeed, to find
the true global minimum we must also let the z-coordinate vary. The resulting 3D problem is significantly harder that the 2D problem, in particular we note that
there is not periodicity in the z-coordinate. However, we still have an ace up our sleeve: the objective function has two mirror planes at $x_1=1/2$ and $x_2 = 1/2$.
BOSS let's you exploit symmetries through objective functions that return an arbitrary number of observations $(X, Y)$. We can write a new method for the `Adatom` class
that does exactly this for the two mirror planes:

In [ ]:
def calc_energy_with_symmetry(self, x):
    """Like calc_energy, but exploits mirror planes to return extra observations. """
    x = np.squeeze(x)
    if self.z_fix is not None and len(x) == 2:
        x = np.append(x, self.z_fix)

    x1 = np.array([1 - x[0], x[1], x[2]])
    x2 = np.array([x[0], 1 - x[1], x[2]])
    x3 = np.array([1 - x[0], 1 - x[1], x[2]])
    X = np.array([x, x1, x2, x3])

    # filter duplicates
    _, inds = np.unique(X.round(decimals=4), axis=0, return_index=True)
    X = X[inds]

    if self.z_fix is not None:
        X = X[:, :2]
    Y = np.ones(len(X)) * self.calc_energy(x)
    return X, Y

Adatom.calc_energy_with_symmetry = calc_energy_with_symmetry

This way, one call to EMT calculator to compute the energy can yield up to 4 new observations. Armed with this new objective function we are better equipped to tackle the full 3D optimization problem, what solution do you obtain? $[1.0, 2.5]$ are reasonable bounds for the z-coordinate.

 **Hints**: You will need to modify both the
bounds and the kernel, the number of iterations might also need to be increased. Look at the `convergence_measures.png` file generated by the post-processing.

In [ ]:
# Switching on symmetry-enhanced acquisitions
adatom = Adatom()

bo = BOMain(
    # please switch on the symmetry in the line below
    adatom.calc_energy_with_symmetry,
    bounds=[[0., 1.], [0., 1.], [1, 2.5]],
    kernel=['stdp', 'stdp', 'rbf'],
    initpts=4,
    iterpts=20,
    acqfn_name='elcb',
)
res = bo.run()

pp = PPMain(res, pp_models=True, pp_acq_funcs=True)
pp.run()

# We can also view the minimum energy configuration
x_glmin = res.select("x_glmin", -1)
y_glmin = res.select("mu_glmin", -1)
print("Best adsorption energy found: %0.3f eV" %y_glmin)
adatom.place(x_glmin)
view(adatom.atoms, viewer='x3d')
